In [2]:
import pdfplumber
import json
import re
from elasticsearch import Elasticsearch
from functools import partial


/Users/devih-21/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Config ignore table when extract_text

In [29]:
def not_within_bboxes(obj, bboxes):
    """Check if the object is in any of the table's bbox."""

    def obj_in_bbox(_bbox):
        """Define objects in box.

        See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404
        """
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)

    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

def filter_tables(page: pdfplumber.page.Page) -> pdfplumber.page.Page:
    if page.find_tables() != []:
        bboxes = [table.bbox for table in page.find_tables()]
        bbox_not_within_bboxes = partial(not_within_bboxes, bboxes=bboxes)

        page = page.filter(bbox_not_within_bboxes)

    return page
  


Create raw data after read from pdf (ignore table)

In [3]:

with pdfplumber.open('s3-userguide.pdf') as pdf:
    # init raw data array
    lines_data = []
    # loop each page with 
    for page_num, page in enumerate(pdf.pages, start=1):
        lines = filter_tables(page).extract_text(layout=True).split('\n')
        for line_num, line in enumerate(lines, start=1):
            lines_data.append({
                "content": line,
                "title": lines[2],
                "section_title": lines[-4]
            })

    with open('raw.json', 'w', encoding='utf-8') as f:
        json.dump(lines_data, f, ensure_ascii=False, indent=4)

NameError: name 'filter_tables' is not defined

Clean, add rule and group raw data to data which can import to ES

In [27]:
with open('raw.json', 'r', encoding='utf-8') as f:
    lines_data = json.load(f)

filtered_lines_data = []
debug = []
current_content = ''
for line_data in lines_data:
    content = line_data['content'].strip()
    if content != '':
        content = re.sub(r'\s+', ' ', content)
        if "API Version 2006-03-01" not in content and "Amazon Simple Storage Service" not in content and content != '':
            if current_content:
              if (content[0] == '-' or content[0] == '•' or content[0].isnumeric()):
                  current_content += '\n' + content
              elif content[0].islower():
                  current_content += ' ' + content
              elif (current_content[-1] != '.'):
                  current_content += ' ' + content
              elif  (current_content[-1] == ',' or current_content[-1] == ':'):
                  current_content += ' ' + content
              elif content[0].islower():
                  current_content += ' ' + content
              else:
                  new = {}
                  new['title'] = re.sub(r'\s+', ' ', line_data['title'].strip())
                  new['content'] = current_content
                  new['section_title'] = line_data['section_title'].strip().split("API Version")[0].strip()
                  if new['section_title'] and "Table of Contents" not in new['content']:
                      filtered_lines_data.append(new)
                  current_content = content
            else:
                current_content = content
                  

with open('output.json', 'w', encoding='utf-8') as f:
    json.dump(filtered_lines_data, f, ensure_ascii=False, indent=4)


Put data to ES

In [ ]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

with open('output.json', 'r') as file:
    data = json.load(file)

for doc in data:
    es.index(index='s3_guide', body=doc)

/var/folders/yz/tvpwz_9d6c56fms92tltys800000gn/T/ipykernel_55600/2731284087.py:8: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.index(index='s3_guide', body=doc)
